In [170]:
import numpy as np

In [171]:
def invert_changed_row(A_inv, x, i):
    l = A_inv.dot(x)
    if l[i] == 0:
        print('not invertable')
        return
    l_i = l[i]
    l[i] = -1
    l = - l / l_i

    n = A_inv.shape[0]
    E = np.eye(n)
    E[:, i] = np.array(l)

    
    A_hat = np.zeros((n, n))
    for k in range(n):
        for j in range(n):
            if k == i:
                A_hat[k, j] = E[k,k] * A_inv[k,j]
                continue
            A_hat[k, j] = E[k,k] * A_inv[k,j] + E[k, i] * A_inv[i, j]
    return A_hat

In [172]:
def simplex_method(c: np.array, x: np.array, B: np.array, A: np.array, b: np.array, verbose: int=False):

    B -= 1
    iter_cnt = 1
    
    if verbose == 2:
        print('\t\tSTARTING SIMPLEX METHOD')
        print('x:\n', x)
        print('B:\n', B+1)

    
    while True:
        A_b = A[:, B]
        if verbose == 2:
            print('\t\tITERATION: ', iter_cnt)
            print('A_b:\n', A_b)
            
        if iter_cnt == 1:
            A_b_inv = np.linalg.inv(A_b)
        else:
            A_b_inv = invert_changed_row(A_b_inv.copy(), A[:, j_0].copy(), k)
        if verbose == 2:
            print('A_b_inv:\n', A_b_inv)
            
        c_b = c[B]
        if verbose == 2:
            print('c:\n', c)
            print('c_b:\n', c_b)

        # potential vector
        u = c_b @ A_b_inv
        if verbose == 2:
            print('u:\n', u)

        # eveluation_vector
        nabla = u @ A - c
        if verbose == 2:
             print('nablas:\n', nabla)
        if (nabla >= -1e-8).all():
            if verbose == 1:
                print('\nmax c: ', c[c!=0] @ x[c!=0])
            return x, B
        
        # first neg element index, z, teta, teta_0
        j_0 = nabla.argmin()
        z = A_b_inv @ A[:,j_0]
        teta = np.array([x[B[j]] / el if el > 0 else float('inf') for j, el in enumerate(z)])
        teta_0 = teta.min()
        if verbose == 2:
            print('j_0:\n', j_0)
            print('z:\n', z)
            print('teta:\n', teta)
            print('teta_0:\n', teta_0)
            
        if teta_0 == float('inf'):
            return "Not Bounded"
        
        k = teta.argmin()
        if verbose == 2:
            print('k:\n', k)
            
        j_star = B[k]
        
        old_B = B.copy()
        B[k] = j_0

        # update current plan
        old_x = x.copy()
        for b_ind, b in enumerate(B):
            x[b] = x[b] - teta_0 * z[b_ind]

        x[j_0] = teta_0
        x[j_star] = 0
        
        if verbose == 2:
            print('New B:\n', B+1)
            print('New x:\n', x)
                    
        iter_cnt += 1

In [173]:
c = np.array([
    1, 1, 0, 0, 0
], dtype=float)

A = np.array([
    [-1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 0, 1]
], dtype=float)

b = np.array([
    1, 3, 2
], dtype=float)

x_start = np.array([
    0, 0, 1, 3, 2
], dtype=float)

B_start = np.array([
    3, 4, 5
])

simplex_method(c.copy(), x_start.copy(), B_start.copy(), A.copy(), b.copy(), False)

(array([3., 2., 2., 0., 0.]), array([2, 0, 1]))

In [174]:
c = np.array([
    3, 2, 0, 0, 0
], dtype=float)

A = np.array([
    [2, 1, 1, 0, 0],
    [2, 3, 0, 1, 0],
    [3, 1, 0, 0, 1]
], dtype=float)

b = np.array([
    18, 42, 24
], dtype=float)

B_start = np.array([
    3, 4, 5
], dtype=int)

x_start = np.array([
    0, 0, 18, 42, 24
], dtype=float)

  
simplex_method(c.copy(), x_start.copy(), B_start.copy(), A.copy(), b.copy(), False)

(array([ 3., 12.,  0.,  0.,  3.]), array([1, 4, 0]))

### First stage of simplex method

In [222]:
def first_stage_simplex(A: np.array, b: np.array, verbose: int=False):
    m = A.shape[0]
    n = A.shape[1]
    if verbose == 2:
        print('m*n:', m, 'x', n)
        print('b:\n', b)
    # make b not negative
    for idx, val in enumerate(b):
        if val < 0:
            A[idx, :] *= -1
            b[idx] *= -1
    
    c_hat = np.ones(m+n) * -1
    c_hat[np.arange(n)] = 0
    A_hat = np.hstack([A, np.eye(m)])
    if verbose == 2:
        print('c_hat:\n', c_hat)
        print('A_hat:\n', A_hat)
    
    # basic allowable plan
    x_hat = np.zeros(n+m)
    x_hat[np.arange(n, n+m)] = b
    B = np.arange(n, n+m)
    if verbose == 2:
        print('x_hat:\n', x_hat)
        print('B:\n', B+1)
    
    
    x, B = simplex_method(c_hat.copy(), x_hat.copy(), B.copy() + 1, A_hat.copy(), b.copy(), verbose=0)
    if verbose == 2:
        print('x:\n', x)
        print('B:\n', B)
    if (abs(x[np.arange(n, m+n)]) > 1e-8).all():
        print('Not compatible')
        return 'Not copatible'
    
    flag = True
    while flag:
        n, m = A.shape
        if (B < n).all():
            return A, b


        # find max index of free varibles
        j_k = 0
        k = 0
        i = 0
        for idx, val in enumerate(B):
            if val >= n:
                if j_k < val:
                    j_k = val
                    k = idx
                    i = n+m - j_k
        if verbose == 2:
            print('j_k:', j_k)
            print('k:', k)
            print('i:', i)

        newbies = set(np.arange(0, n)) - set(B)
        if verbose == 2:
            print('newbies:\n', newbies)
    
        l = []
        A_hat_b_inv = np.linalg.pinv(A_hat[:, B])
        if verbose == 2:
            print('A_hat_b:\n', A_hat[:, B])
            print('A_hat_b_inv:\n', A_hat_b_inv)

        repeat_cnt = 0
        for idx, each in enumerate(newbies):
            l.append(A_hat_b_inv @ A[:, each][:, np.newaxis])
            if abs(l[idx][k]) < 1e-8:
                repeat_cnt += 1
            else:
                B[k] = each


        if verbose == 2:
            print('l:\n', l)

        # deleting extras
        if repeat_cnt == len(l):
            A = np.delete(A, i, 0)
            A_hat = np.delete(A_hat, i, 0)
            b = np.delete(b, i, 0)
            B = np.delete(B, k, 0)
        else:
            flag = False
    return A, b
    

In [223]:
A = np.array([
    [1, 1, 1, 1, 1],
    [2, 2, 2, 2, 2],
    [3, 3, 3, 3, 3],
    [4, 4, 4, 4, 4]
    

], dtype=float)

b = np.array([0, 0, 0, 0], dtype=float)

answer = first_stage_simplex(A, b, verbose=2)
print()

if type(answer) == 'tuple':
    print('Not compatible')
else:
    print('Have valid plan:')
    print('A: ', answer[0])
    print('b: ', answer[1])

m*n: 4 x 5
b:
 [0. 0. 0. 0.]
c_hat:
 [ 0.  0.  0.  0.  0. -1. -1. -1. -1.]
A_hat:
 [[1. 1. 1. 1. 1. 1. 0. 0. 0.]
 [2. 2. 2. 2. 2. 0. 1. 0. 0.]
 [3. 3. 3. 3. 3. 0. 0. 1. 0.]
 [4. 4. 4. 4. 4. 0. 0. 0. 1.]]
x_hat:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
B:
 [6 7 8 9]
x:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
B:
 [0 6 7 8]
j_k: 8
k: 3
i: 1
newbies:
 {1, 2, 3}
A_hat_b:
 [[1. 0. 0. 0.]
 [2. 1. 0. 0.]
 [3. 0. 1. 0.]
 [4. 0. 0. 1.]]
A_hat_b_inv:
 [[ 1.00000000e+00  2.22044605e-16  2.77555756e-17 -2.22044605e-16]
 [-2.00000000e+00  1.00000000e+00  2.22044605e-16  2.77555756e-16]
 [-3.00000000e+00 -9.43689571e-16  1.00000000e+00  7.28583860e-16]
 [-4.00000000e+00 -1.27675648e-15 -2.42861287e-16  1.00000000e+00]]
l:
 [array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]]), array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]]), array([[1.00000000e+00],
       [1.11022302e-15],
       [1.77635684e-15],
       [8.88178420e-16]